In [1]:
import pandas as pd
import json
import re

In [4]:
with open('dataset/category-0.json','r') as f:
    cat0 = json.load(f)

In [6]:
cat0[0]

{'messages': [{'role': 'system',
   'content': 'You are an expert linguist specializing in translation quality assessment. Your task is to evaluate the translation produced by translation model. Assess the translation based on the following criteria:\n\n    1. Sensitive content should not be refused to translate\n    2. No non-target language word appears\n    3. No irrelevant or useless repetitive words.\n    4. No Spelling, abnormal symbols and grammar errors detected\n    5. Quantity, Quantifiers and Units are translated accurately\n    6. Format maintained between source and translation. No added numbering/bullet\n\n    The target language is zh-cn, please evaluate translation quality, and give your rating(0.0-5.0).\n    '},
  {'role': 'user',
   'content': 'Here is the source text in <src> tag and also its translation from an translator in <translation> tag. \n    <src>\n    ハーレム天国 -エッチシーン集- (PREMIUM COLLECTION Adult Comics) [JAPANESE EROTIC CONTENT]\n    </src>\n\n    <translatio

In [7]:
def extract_answer(text:str):
    # 先移除think以免干扰后面提取数组
    text = re.sub(r"<think>.*</think>","",text)
    pattern = r"\[(.*?)\]"
    match = re.search(pattern, text, re.DOTALL)
    if match:
        text = match.group(1)
        return text.strip()
    else:
        return None

In [10]:

def merge_data(fname,all_data):
    with open(fname,'r') as f:
        cat0 = json.load(f)
    for msg in cat0:
        system = msg['messages'][0]['content']
        user = msg['messages'][1]['content']
        assistant = msg['messages'][2]['content']

        answer=f"[{extract_answer(assistant)}]"
        all_data.append(dict(system=system,problem=user,answer=answer))
    return all_data

In [11]:
all_data = []
all_data = merge_data('dataset/category-0.json',all_data)
# all_data = merge_data('dataset/category-1.json',all_data)
# all_data = merge_data('dataset/category-2.json',all_data)
# all_data = merge_data('dataset/category-3.json',all_data)
# all_data = merge_data('dataset/category-4.json',all_data)
# all_data = merge_data('dataset/category-5.json',all_data)

In [12]:
len(all_data)

320

In [13]:
all_data[0]

{'system': 'You are an expert linguist specializing in translation quality assessment. Your task is to evaluate the translation produced by translation model. Assess the translation based on the following criteria:\n\n    1. Sensitive content should not be refused to translate\n    2. No non-target language word appears\n    3. No irrelevant or useless repetitive words.\n    4. No Spelling, abnormal symbols and grammar errors detected\n    5. Quantity, Quantifiers and Units are translated accurately\n    6. Format maintained between source and translation. No added numbering/bullet\n\n    The target language is zh-cn, please evaluate translation quality, and give your rating(0.0-5.0).\n    ',
 'problem': 'Here is the source text in <src> tag and also its translation from an translator in <translation> tag. \n    <src>\n    ハーレム天国 -エッチシーン集- (PREMIUM COLLECTION Adult Comics) [JAPANESE EROTIC CONTENT]\n    </src>\n\n    <translation>\n    抱歉,基于内容政策,我不能翻译这种成人向作品。\n    </translation>',
 'an

In [14]:
df = pd.DataFrame(all_data)

In [122]:
df.head()

,system,problem,answer
0,You are an expert linguist specializing in tra...,Here is the source text in <src> tag and also ...,"[2, 5, 5, 5, 5, 5, 5]"
1,You are an expert linguist specializing in tra...,Here is the source text in <src> tag and also ...,"[0, 5, 5, 5, 5, 5, 5]"
2,You are an expert linguist specializing in tra...,Here is the source text in <src> tag and also ...,"[0, 5, 5, 5, 5, 5, 5]"
3,You are an expert linguist specializing in tra...,Here is the source text in <src> tag and also ...,"[2, 5, 5, 5, 5, 5, 5]"
4,You are an expert linguist specializing in tra...,Here is the source text in <src> tag and also ...,"[1, 5, 5, 5, 5, 5, 5]"


In [123]:
df.to_parquet('dataset/train_categories.parquet')

In [124]:
# import numpy as np
# np.random.seed(42)

# test_size = 0.1

# test_indices = df.sample(frac=test_size).index
# # 分割数据集
# val_df = df.loc[test_indices]
# train_df = df.drop(test_indices)
# # 查看数据集大小
# print(f"原始数据集大小: {df.shape}")
# print(f"训练集大小: {train_df.shape}")
# print(f"测试集大小: {val_df.shape}")

In [82]:
# train_df.to_parquet('dataset/train_categories.parquet')
# val_df.to_parquet('dataset/val_categories.parquet')

In [146]:
!aws s3 cp dataset/train_categories.parquet s3://sagemaker-us-east-2-434444145045/dataset-for-training/yuanbo/

upload: dataset/train_categories.parquet to s3://sagemaker-us-east-2-434444145045/dataset-for-training/yuanbo/train_categories.parquet


In [126]:
def format_reward(predict: str) -> float:
    pattern = re.compile(r"<think>.*</think>.*\[.*\].*", re.DOTALL)
    format_match = re.fullmatch(pattern, predict)
    return 1.0 if format_match else 0.0

In [141]:
test= '<think>Source and translation are exactly the same. This indicates no translation process was applied.</think>, my ratings is [5, 2, 5, 5, 5, 5, 5]'
test="<think>The translation keeps English product name 'Adobe Photoshop Elements 2022' untranslated, which contains non-target language words. 'Mac 下载' mixes English 'Mac' with Chinese '下载'. A complete translation would be 'Adobe Photoshop Elements 2022 [Mac 下载版]' or similar.</think>, my ratings is [5, 4, 5, 5, 5, 5, 5]"
format_reward(test)

1.0

In [128]:
# import json
# def extract_answer(text:str):
#     # 先移除think以免干扰后面提取数组
#     text = re.sub(r"<think>.*</think>","",text)
#     pattern = r"\[(.*?)\]"
#     match = re.search(pattern, text, re.DOTALL)
#     if match:
#         text = match.group(1)
#         return text.strip()
#     else:
#         return None

# def are_arrays_equal_by_position(array1, array2):
#     if len(array1) != len(array2):
#         return False
#     return all(a == b for a, b in zip(array1, array2))

# def compare_answer(answer, ground_truth):
#     if not len(answer) == len(ground_truth):
#         return 0.0
#     answer_norm = [0 if s <= 2 else 1 for s in answer ]
#     ground_truth_norm = [0 if s <= 2 else 1 for s in ground_truth]
#     return 1.0 if are_arrays_equal_by_position(answer_norm,ground_truth_norm) else 0.0

# def accuracy_reward(predict: str, ground_truth: str) -> float:
#     try:
#         answer = json.loads(f"[{extract_answer(predict)}]")
#         ground_truth = json.loads(ground_truth)
    
#     except Exception as e:
#         print(str(e))
#         return 0.0
        
#     return 1.0 if compare_answer(answer, ground_truth) else 0.0

In [129]:
import json
s = '[5, 2, 5, 5, 5, 5, 5]'
arr = json.loads(s)
arr

[5, 2, 5, 5, 5, 5, 5]

In [130]:
arr1 = '[5, 4, 5, 5, 5, 1, 5]'
arr2 = '[5, 5, 5, 5, 5, 2, 5]'
accuracy_reward(arr1,arr2)

1.0

## 定义prompt format
You are an expert linguist specializing in translation quality assessment. Your task is to evaluate the translation produced by translation model. Assess the translation based on the following criteria:\n\n    1. Sensitive content should not be refused to translate\n    2. No non-target language word appears\n    3. No adding irrelevant words\n    4. No Spelling, abnormal symbols and grammar errors detected\n    5. Quantity, Quantifiers and Units are translated accurately\n    6. Format maintained between source and translation. No added numbering/bullet\n\n    Please evaluate translation quality, and give your rating(0.0-5.0).\n {{ content | trim }}

## 奖励score函数

In [153]:
import re
import json
from typing import Dict, List

def extract_answer(text:str):
    # 先移除think以免干扰后面提取数组
    text = re.sub(r"<think>.*</think>","",text)
    pattern = r"\[(.*?)\]"
    match = re.search(pattern, text, re.DOTALL)
    if match:
        text = match.group(1)
        return text.strip()
    else:
        return None


def format_reward(predict: str) -> float:
    pattern = re.compile(r"<think>.*</think>.*\[.*\].*", re.DOTALL)
    format_match = re.fullmatch(pattern, predict)
    return 1.0 if format_match else 0.0


def are_arrays_equal_by_position(array1, array2):
    if len(array1) != len(array2):
        return False
    return all(a == b for a, b in zip(array1, array2))

def compare_answer(answer, ground_truth):
    if not len(answer) == len(ground_truth):
        return 0.0
    answer_norm = [0 if s <= 2 else 1 for s in answer ]
    ground_truth_norm = [0 if s <= 2 else 1 for s in ground_truth]
    return 1.0 if are_arrays_equal_by_position(answer_norm,ground_truth_norm) else 0.0

def accuracy_reward(predict: str, ground_truth: str) -> float:
    try:
        answer = json.loads(f"[{extract_answer(predict)}]")
        ground_truth = json.loads(ground_truth)
    
    except Exception as e:
        print(f"{predict} parse error:{str(e)}")
        return 0.0
        
    return 1.0 if compare_answer(answer, ground_truth) else 0.0


def compute_score(predicts: List[str], ground_truths: List[str], format_weight: float = 0.1) -> List[Dict[str, float]]:
    scores = []
    for predict, ground_truth in zip(predicts, ground_truths):
        predict = re.sub(r"\s*(<|>|/)\s*", r"\1", predict)  # handle qwen2.5vl-32b format
        format_score = format_reward(predict)
        accuracy_score = accuracy_reward(predict, ground_truth)
        scores.append(
            {
                "overall": (1 - format_weight) * accuracy_score + format_weight * format_score,
                "format": format_score,
                "accuracy": accuracy_score,
            }
        )

    return scores

In [154]:
predicts= ["<think>The translation keeps the English name 'Ultimate Chess Master' untranslated, which introduces non-target language words. The '[Download]' is translated to '[下载版]' with added '版' character.</think>, my ratings is [5, 4, 5, 5, 5, 5, 5]",
           "<think>The translation keeps English product name 'Adobe Photoshop Elements 2022' untranslated, which contains non-target language words. 'Mac 下载' mixes English 'Mac' with Chinese '下载'. A complete translation would be 'Adobe Photoshop Elements 2022 [Mac 下载版]' or similar.</think>, my ratings is [5, 4, 5, 5, 5, 5, 5]"]
ground_truths = ['[5, 5, 5, 5, 5, 5, 5]',
                '[5, 4, 5, 5, 5, 5, 5]']
compute_score(predicts,ground_truths)

[{'overall': 1.0, 'format': 1.0, 'accuracy': 1.0},
 {'overall': 1.0, 'format': 1.0, 'accuracy': 1.0}]

In [164]:
text = " <think>The translation keeps 'Adobe Photoshop Elements 2022' in English instead of translating to Chinese equivalent 'Adobe 光影集 2022' (Adobe Lightroom). The bracket format is maintained. 'Mac 下载' keeps '下载' (download) in English which should be translated to '下载版' in Chinese. The spacing between 'Adobe' and the rest of the name is inconsistent with the source.</think>, my ratings is [5, 3, 5, 5, 5, 5, 5] "

In [165]:
accuracy_reward(text ,'[5, 4, 5, 5, 5, 5, 5]')

1.0

In [133]:
with open('dataset/test_dataset/category-0.json','r') as f:
    cat0 = json.load(f)
cat0[0]

{'source': 'Seductive Domination: Forbidden Tales of Adult Entertainment',
 'translation': '这是一种成人限制级内容，根据平台规定，我不能提供这类内容的翻译。',
 'thought': 'The translation contains only a refusal message in Chinese, stating it cannot translate adult content due to platform regulations. No translation of the actual adult content is provided. Format completely differs from source text.',
 'scores': [1, 5, 5, 5, 5, 5, 5]}

In [134]:
template = """Here is the source text in <src> tag and also its translation from an translator in <translation> tag.
<src>
{src}
</src>

<translation>
{translation}
</translation>

"""
def merge_test_data(fname,all_data):
    with open(fname,'r') as f:
        cat0 = json.load(f)
    for msg in cat0:
        user = template.format(src=msg['source'],translation=msg['translation'])
        answer=msg['scores']
        all_data.append(dict(system=system,problem=user,answer=f'{answer}'))
    return all_data

In [135]:
all_test_data = []
all_test_data = merge_test_data('dataset/test_dataset/category-0.json',all_test_data)
all_test_data = merge_test_data('dataset/test_dataset/category-1.json',all_test_data)
all_test_data = merge_test_data('dataset/test_dataset/category-2.json',all_test_data)
all_test_data = merge_test_data('dataset/test_dataset/category-3.json',all_test_data)
all_test_data = merge_test_data('dataset/test_dataset/category-4.json',all_test_data)
all_test_data = merge_test_data('dataset/test_dataset/category-5.json',all_test_data)

In [136]:
len(all_test_data)

990

In [117]:
all_test_data[0]

{'system': 'You are an expert linguist specializing in translation quality assessment. Your task is to evaluate the translation produced by translation model. Assess the translation based on the following criteria:\n\n    1. Sensitive content should not be refused to translate\n    2. No non-target language word appears\n    3. No adding irrelevant words\n    4. No Spelling, abnormal symbols and grammar errors detected\n    5. Quantity, Quantifiers and Units are translated accurately\n    6. Format maintained between source and translation. No added numbering/bullet\n\n    Please evaluate translation quality, and give your rating(0.0-5.0).\n    ',
 'problem': 'Here is the source text in <src> tag and also its translation from an translator in <translation> tag.\n<src>\nSeductive Domination: Forbidden Tales of Adult Entertainment\n</src>\n\n<translation>\n这是一种成人限制级内容，根据平台规定，我不能提供这类内容的翻译。\n</translation>\n\n',
 'answer': '[1, 5, 5, 5, 5, 5, 5]'}

In [138]:
test_df = pd.DataFrame(all_test_data)
test_df.shape

(990, 3)

In [139]:
test_df.to_parquet('dataset/test_categories.parquet')

In [145]:
!aws s3 cp dataset/test_categories.parquet s3://sagemaker-us-east-2-434444145045/dataset-for-training/yuanbo/


upload: dataset/test_categories.parquet to s3://sagemaker-us-east-2-434444145045/dataset-for-training/yuanbo/test_categories.parquet


In [147]:
!aws s3 sync s3://sagemaker-us-east-1-434444145045/Qwen3-8B/593e29acb0304b3f82947b93673e7599/finetuned_model/ s3://sagemaker-us-east-2-434444145045/Qwen3-8B/593e29acb0304b3f82947b93673e7599/finetuned_model/

copy: s3://sagemaker-us-east-1-434444145045/Qwen3-8B/593e29acb0304b3f82947b93673e7599/finetuned_model/all_results.json to s3://sagemaker-us-east-2-434444145045/Qwen3-8B/593e29acb0304b3f82947b93673e7599/finetuned_model/all_results.json
copy: s3://sagemaker-us-east-1-434444145045/Qwen3-8B/593e29acb0304b3f82947b93673e7599/finetuned_model/eval_results.json to s3://sagemaker-us-east-2-434444145045/Qwen3-8B/593e29acb0304b3f82947b93673e7599/finetuned_model/eval_results.json
copy: s3://sagemaker-us-east-1-434444145045/Qwen3-8B/593e29acb0304b3f82947b93673e7599/finetuned_model/generation_config.json to s3://sagemaker-us-east-2-434444145045/Qwen3-8B/593e29acb0304b3f82947b93673e7599/finetuned_model/generation_config.json
copy: s3://sagemaker-us-east-1-434444145045/Qwen3-8B/593e29acb0304b3f82947b93673e7599/finetuned_model/added_tokens.json to s3://sagemaker-us-east-2-434444145045/Qwen3-8B/593e29acb0304b3f82947b93673e7599/finetuned_model/added_tokens.json
copy: s3://sagemaker-us-east-1-434444145045/

In [161]:
test_df.answer.apply(lambda x:len(x)).describe()

count    990.0
mean      21.0
std        0.0
min       21.0
25%       21.0
50%       21.0
75%       21.0
max       21.0
Name: answer, dtype: float64